In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
  
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5319708948942509158
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14509932544
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5512374829901747592
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]
Default GPU Device: /device:GPU:0


In [ ]:
import platform
import tensorflow as tf

if tf.test.is_built_with_cuda() == False:
    print('ARBEIT OHNE GPU')

print(platform.machine(), '\n')
print(platform.version(), '\n')
print(platform.platform(), '\n')
print(platform.uname(), '\n')
print(platform.system(), '\n')
print(platform.processor())

x86_64 

#1 SMP Sat Jun 5 09:50:34 PDT 2021 

Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic 

uname_result(system='Linux', node='e783529f9eb9', release='5.4.104+', version='#1 SMP Sat Jun 5 09:50:34 PDT 2021', machine='x86_64', processor='x86_64') 

Linux 

x86_64


In [ ]:
import pandas as pd
import numpy as np
import json
import re
import string

df = pd.read_csv('/content/drive/MyDrive/StumbleUpon_translated.csv')
df = df.rename(columns={'boilerplate':'text', 'label':'label_id'})
df_drive = pd.read_csv('/content/drive/MyDrive/Drive-df.csv')

# a little bit cleaning
labels = pd.DataFrame({'label_id': [0,1], 'label_text': ['Non-Evergreen','Evergreen']})
df = df.join(labels.set_index('label_id')[['label_text']], on='label_id')
df_drive = df_drive.join(labels.set_index('label_id')[['label_text']], on='label_id')

#rename index
new_index=list(range(len(df)+1, len(df)+1+len(df_drive)))
new_index = pd.DataFrame({'new_index': new_index})
df_drive.rename(index=new_index['new_index'],inplace=True)

#naming sources
df['source'] = 'StumbleUpon'
df_drive['source'] = 'Drive'

del df['Unnamed: 0']
del df_drive['Unnamed: 0']
del df['urlid']

#delete rows with null
df = df.loc[~df['text_content'].isnull()]

# append both stumbleupon and drive
df = df.append(df_drive)

# reset index
df.reset_index(inplace=True)

del df['index']

In [ ]:
# combine title and text
def concat_title_text(df):
  liste = []
  for i in range(len(df)):
    liste.append(df.title[i] + df.text_content[i])
  
  return liste

df['text_content'] = concat_title_text(df)

#### Data Preprocessing
There are two methods:
- Bag of Words Model (create a dic of all words and their frequency, remove high frequence words (or filler words)) 
- Term frequency- invers document frequency (tf-idf) (different notebook)

#### Splitting text into title and textcontent

#### NOTE: 

Splitting the data into title and body could improve the accuracy upto 20%. In every case the fullbodytext has been performing worse than this approach below. 

Thus, the meaning in title are heavy weighted if it comes both to detect evergreens and non evergreens (and also multi-classes).  

#### Deleting symbolics and useless characters


#### NOTES:
We could observe that after the data cleaning, based on the regex, have slighty increased the accuracy of the binary clf.

In addition after regex cleaning, the model performed way better in detail (Time complexity, CF-Matrix, multiclass clf, etc.). 

In [ ]:
#get rid of further symbols 
df['text_content'] = df['text_content'].replace({'[»„‘’“”…]': ' '}, regex=True)

# get rid of urls
df['text_content'] = df['text_content'].replace({r"https?://\S+|www\.\S+": ' '}, regex=True)

#get not identified unicode
df['text_content'] = df['text_content'].replace('[\u0080-\uffff]w{1-3}', " ", regex=True)

# replaces all stringw which are unicodes (\u2009 \a0x) and also removes bashes
df['text_content'] = df['text_content'].replace({r"[^\x00-\x7F\w{1,3}]+": ' '}, regex=True)

#### Delete Stopwords from Matrix

##### NOTE:

We could observe that deleting stopwords enhanced performance results without jeopardizing the accuarcy of the model.
In some cases the accuracy got a little bit worse, for instance the LinearSVC performed better on evergreen classification without deleting stopwords.
However, in most models the (especially the multi clf) benefits of the stopword removal. 

### In this NB Wordgroups are not included. Since they enhance runtime and do not improve accuracy significantly enough

In [ ]:
# load own stopword list generated in the EDA and add it to the old stopwords
with open('/content/drive/MyDrive/stopwords_nltk+eda.txt', 'r', encoding="utf-8") as f:
    german_stop_words = f.read().split(',')

# detect and remove stopwords based on german_stop_words
def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in german_stop_words]
  
    return " ".join(text)

# remove all stopwords
df['text_content'] = df['text_content'].map(remove_stopwords) 

# MODEL

In [ ]:
from sklearn.model_selection  import train_test_split

# train test splitting
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)

# start index at 0
df_train.reset_index(inplace=True)
df_test.reset_index(inplace=True)

# BERT for multilingual uncased

In [ ]:
from transformers import AutoTokenizer, TFAutoModel

#Downloading the tokenizer and the Albert model for fine tuning

tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-uncased')
bert=TFAutoModel.from_pretrained('bert-base-multilingual-uncased')

Some layers from the model checkpoint at bert-base-multilingual-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# max length of berttokenizer  is 512
max_length=512

#creating mask for tokens
Xids_train=np.zeros((df_train.shape[0],max_length))
Xmask_train=np.zeros((df_train.shape[0],max_length))
y_train=np.zeros((df_train.shape[0],1))

#creating mask for tokens
Xids_test=np.zeros((df_test.shape[0],max_length))
Xmask_test=np.zeros((df_test.shape[0],max_length))

In [ ]:
# for i (which is the index of article) and sequence (which is the text_content)
for i,sequence in enumerate(df_train['text_content']):
    #Tokenize and prepare for the model
    tokens=tokenizer.encode_plus(sequence,max_length=max_length,padding='max_length',add_special_tokens=True,
                           truncation=True,return_token_type_ids=False,return_attention_mask=True,
                           return_tensors='tf')
    # tokens generated
    Xids_train[i,:] = tokens['input_ids']
    # 0 shows a word which will be filled. 1 a word which will be tokened uniquely
    Xmask_train[i,:] = tokens['attention_mask']
    #gets labels into array
    y_train[i,0] = df_train.loc[i,'label_id']
    
for i,sequence in enumerate(df_test['text_content']):
    tokens=tokenizer.encode_plus(sequence,max_length=max_length,padding='max_length',add_special_tokens=True,
                           truncation=True,return_token_type_ids=False,return_attention_mask=True,
                           return_tensors='tf')
    Xids_test[i,:] = tokens['input_ids']
    Xmask_test[i,:] = tokens['attention_mask']

In [ ]:
# put results of sells above together
dataset=tf.data.Dataset.from_tensor_slices((Xids_train,Xmask_train,y_train))

# change result as one dict except of labels staying as tensors
def map_func(input_ids,mask,labels):
    return {'input_ids':input_ids,'attention_mask':mask},labels

# apply on all values
dataset=dataset.map(map_func)
# shuffling and define 64 articles in 1 batch result for training
dataset=dataset.shuffle(100000).batch(64).prefetch(1000)

# getting numbers of the batches
DS_size=len(list(dataset))

# batch val and train ratio
train=dataset.take(round(DS_size*0.85))
val=dataset.skip(round(DS_size*0.85))

In [ ]:
# do the same combining for test_data
dataset_test=tf.data.Dataset.from_tensor_slices((Xids_test,Xmask_test))

def map_func(input_ids,mask):
    return {'input_ids':input_ids,'attention_mask':mask}

dataset_test=dataset_test.map(map_func)
# batching it to or the predictions will be multiplied by the shape
dataset_test=dataset_test.batch(64).prefetch(1000)

In [ ]:
# printing one train-batch
list(train)[0]

({'attention_mask': <tf.Tensor: shape=(64, 512), dtype=float64, numpy=
  array([[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 0., 0., 0.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 0., 0., 0.]])>,
  'input_ids': <tf.Tensor: shape=(64, 512), dtype=float64, numpy=
  array([[  101., 93354., 10234., ..., 68859., 21721.,   102.],
         [  101., 77423., 28561., ...,     0.,     0.,     0.],
         [  101., 19047., 87029., ..., 47822., 73293.,   102.],
         ...,
         [  101., 93135., 12354., ..., 44673., 59665.,   102.],
         [  101., 12537., 87951., ..., 10143., 85351.,   102.],
         [  101., 10514., 11248., ...,     0.,     0.,     0.]])>},
 <tf.Tensor: shape=(64, 1), dtype=float64, numpy=
 array([[0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],


In [ ]:
#save model
from __future__ import absolute_import, division, print_function
import os

tf.__version__

checkpoint_path = "/content/model_pretrain/Colab Notebooks"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
import tensorflow as tf 

def evergreen_model():
  input_ids=tf.keras.layers.Input(shape=(max_length,),name='input_ids',dtype='int32')
  input_mask=tf.keras.layers.Input(shape=(max_length,),name='attention_mask',dtype='int32')
  
  # keras layers suggestion based on BERT textclf
  embedding=bert(input_ids,attention_mask=input_mask)[0]
  x=tf.keras.layers.GlobalMaxPool1D()(embedding)
  x=tf.keras.layers.GlobalAveragePooling1D()(embedding)
  x=tf.keras.layers.BatchNormalization()(x)
  x=tf.keras.layers.Dense(128,activation='relu')(x)
  x=tf.keras.layers.Dropout(0.3)(x)
  x=tf.keras.layers.Dense(64,activation='relu')(x)
  output=tf.keras.layers.Dense(1,activation='sigmoid')(x)
  
  model=tf.keras.Model(inputs=[input_ids,input_mask],outputs=output)
  
  model.layers[2].trainable=False

  return model

In [ ]:
model = evergreen_model()

model.summary()

Model: "model_27"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 167356416   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
global_average_pooling1d_27 (Gl (None, 768)          0           tf_bert_model[27][0]      

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer='adam',metrics=[tf.keras.metrics.AUC()])


#model.fit(train,validation_data=val,epochs=1, callbacks=[cp_callback])


In [ ]:
model_loaded = evergreen_model()

model_loaded.load_weights(checkpoint_path)

In [ ]:
predictions=model_loaded.predict(dataset_test)

In [ ]:
df_test['predicted_label'] = predictions

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## Calculating best Threshhold + Eval
Source: 
https://machinelearningmastery.com/threshold-moving-for-imbalanced-classification/#:%7E:text=Optimal%20Threshold%20for%20Precision%2DRecall%20Curve,-Unlike%20the%20ROC&text=Recall%20is%20calculated%20as%20the,is%20the%20same%20as%20sensitivity.

In [ ]:
from sklearn.metrics import precision_recall_curve
from numpy import argmax

# calculate pr curve
precision, recall, thresholds = precision_recall_curve(df_test['label_id'], df_test['predicted_label'])

# convert to f score
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = argmax(fscore)
print('Best Threshold based on preds =%f, F-Score=%.3f' % (thresholds[ix], fscore[ix]))
print('The Second Best Threshold after observing =%f, F-Score=%.3f' % (thresholds[455], fscore[455]))
print('The Third Best Threshold after observing =%f, F-Score=%.3f' % (thresholds[454], fscore[454]))

Best Threshold based on preds =0.656491, F-Score=0.801
The Second Best Threshold after observing =0.565859, F-Score=0.800
The Third Best Threshold after observing =0.505386, F-Score=0.799


In previous predictions we could notice that the best threshhold is between 0.505-0.59
Because of the pred_labels we have ha significant jump from 0.505 to 0.509. Thus, the 'best' threshold appears to be 0.65. 

Therefore, we take the average between the third and second best.




In [ ]:
threshold = (thresholds[454]+thresholds[455])/2

print('New Best Threshold after observing =%f' % (threshold))

New Best Threshold after observing =0.535622


In [ ]:
# show predictions for nonevergreens
tmp_negative = df_test[df_test.label_id == 0][['label_id', 'predicted_label']]

print('Anzahl der true_labels für 0: ', len(tmp_negative), '\n')

print('Anzahl der pred_labels für t: ', len(tmp_negative[tmp_negative.predicted_label < threshold]), '\n')

print('Accuracy für gegeben threshold t: ', (len(tmp_negative[tmp_negative.predicted_label < threshold])/len(tmp_negative)*100), '%')

Anzahl der true_labels für 0:  481 

Anzahl der pred_labels für t:  411 

Accuracy für gegeben threshold t:  85.44698544698545 %


In [ ]:
# show predictions for evergreens
tmp_positive = df_test[df_test.label_id == 1][['label_id', 'predicted_label']]

print('Anzahl der true_labels für 0: ', len(tmp_positive), '\n')

print('Anzahl der pred_labels für t > 0.5: ', len(tmp_positive[tmp_positive.predicted_label > threshold]), '\n')

print('Accuracy für gegeben threshold t: ', (len(tmp_positive[tmp_positive.predicted_label > threshold])/len(tmp_positive))*100, '%')

Anzahl der true_labels für 0:  350 

Anzahl der pred_labels für t > 0.5:  280 

Accuracy für gegeben threshold t:  80.0 %


In [ ]:
# all this should be 0
tmp_0 = df_test[df_test.predicted_label < threshold][['label_id', 'predicted_label']]
# right preds
right_pred_0 = len(tmp_0[tmp_0.label_id == 0])

# all this should be 1
tmp_1 = df_test[df_test.predicted_label > threshold][['label_id', 'predicted_label']]
# right pred
right_pred_1 = len(tmp_1[tmp_1.label_id == 1])

#print accuracy on not seen
print('Accuracy in %:', (right_pred_0+right_pred_1)/len(df_test)*100)

Accuracy in %: 83.15282791817089
